# Assignment 2 - Data from the web

## Data scraping

## 1. Obtain all the data

In [1]:
import requests
from bs4 import BeautifulSoup

The first thing to do is to get the base URL (basically looking at the address bar of the browser).
We need a second variable INDEX_PARAM because this page is using frames, and we'll need the base address to access them.

In [2]:
BASE_URL = "http://isa.epfl.ch/imoniteur_ISAP/%21gedpublicreports.htm"
INDEX_PARAM = "ww_i_reportmodel=133685247"

Then let's fetch the parameters names, but as it is a bit tricker, BeautifulSoup will be a great help for this (and also some manual html inspection).

In [3]:
r = requests.get(BASE_URL + '?' + INDEX_PARAM)
soup = BeautifulSoup(r.text, 'html.parser')


#print(soup)

for frame in soup.find_all('frame'):
    if frame['name'] == 'toc':
        r = requests.get("http://isa.epfl.ch/imoniteur_ISAP/" + frame['src'])
        print(frame['src'])
# The content of the frame

# Print slects
soup = BeautifulSoup(r.text, 'html.parser')
query = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=-1&ww_i_reportModel=133685247'
    


!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247


and finally a nice function to easily query the EPFL page and get the data from it.

In [14]:
def query_epfl(section, startYear, endYear, semesterType, semesterNumber, semesterSeason):
    # Print slects
    soup = BeautifulSoup(r.text, 'html.parser')
    query = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=-1&ww_i_reportModel=133685247'


    for in_put in soup.findAll('input'):
        #print (in_put['name'])
        if in_put.text == "xls":
        #print(in_put['value'])
            query = query + '&' + in_put['name'] + '=' + in_put['value']
     
    for select in soup.findAll('select'):
    #print (select['name'])
        for t in select.findAll('option'):
        #print(t)
            if t.text == section:
            #print(t['value'])
                query = query + '&' + select['name'] + '=' + t['value']
            if t.text == str(startYear) + '-' + str(endYear):
                query = query + '&' + select['name'] + '=' + t['value']
            if t.text == semesterType + " " + str(semesterNumber):
                query = query + '&' + select['name'] + '=' + t['value']
            if t.text == "Semestre " + semesterSeason:
                query = query + '&' + select['name'] + '=' + t['value']
    return query


In [15]:
import pandas as pd
import numpy as np

def query_to_df(section, startYear, endYear, semesterType, semesterNumber, semesterSeason):
    queryString = query_epfl(section, startYear, endYear, semesterType, semesterNumber, semesterSeason)
    table = BeautifulSoup(requests.get(queryString).text, 'html.parser')
    data = []
    for row in table.findAll('tr'):
        cols = row.find_all('td')
        cols = [ele.text.strip().replace('\xa0', ' ') for ele in cols]
        data.append([ele for ele in cols if ele])
    return pd.DataFrame(data[3:]) #We take data from index 3 and up, because that's where the first student record is.

We can now start using those functions to collect data. Example below

In [17]:
df = query_to_df("Informatique", 2007,2008, "Bachelor semestre", 1, "d'automne")
print(df) 

           0                                    1        2       3
0   Monsieur                           Adami Marc  Présent  179181
1     Madame                         Adamow Chloé  Présent  178494
2   Monsieur                          Adouch Zyad  Présent  180553
3     Madame                     Allemann Aurélie  Présent  175314
4   Monsieur                         Andres David  Présent  179393
5   Monsieur                        Arroyo Alvaro  Présent  181601
6   Monsieur                       Beretta Davide  Présent  175330
7     Madame                       Bodinier Manon  Présent  174594
8     Madame                       Bornoz Pauline  Présent  179459
9   Monsieur                     Brandinu Laurent  Présent  178674
10  Monsieur                       Chuard Nicolas  Présent  179036
11  Monsieur                         Cudré Yanouk  Présent  178750
12    Madame                       Djeddou Anissa  Présent  160856
13  Monsieur                       Encontre Henri  Présent  18

In [18]:
#sectionList = ["Architecture", "Chimie et génie chimique","Cours de mathématique spéciales","EME (EPFL Middle East)","Génie civil","Génie mécanique","Génie électrique et électronique","Humanités digitales","Informatique", "Ingénierie financière", "Management de la technologie","Mathématiques", "Microtechnique", "Physique", "Science et génie des matériaux","Sciences et ingénierie de l'environnement", "Sciences et technologies du vivant","Section FCUE", "Systèmes de communication"]
#This dates from back when I thought we had to query every single section. It was the best of time, it was the worst of times.

We're now going to collect and merge every single semester of every single year. We'll append columns "year", "semester".

Together with a student's sciper, we'll use this to make unique indexes. We could've used the name but it's possible that two students with the same name would be in the same class in the same year and semestre. 

In [68]:
frames = []
for year in range(2007, 2017):
    for semester in range (1, 7):
        if(semester % 2 == 0 ):
            season="de printemps"
        else:
            season="d'automne"
        df = query_to_df("Informatique", year,year+1, "Bachelor semestre", semester, season)
        #df.columns = ["civility", "name", "status", "sciper"]
        df['year'] = year
        df['semester'] = semester
        #df.set_index(['sciper', 'year', 'semester'])
        frames.append(df)
result = pd.concat(frames)

We make sure the index is unique

In [47]:
print(df.index.is_unique)

True


In [70]:
from IPython.display import display
display(result)

,0,1,2,3,year,semester,4,5,6
0,Monsieur,Arévalo Christian,Présent,169569,2007,1,NaN,NaN,NaN
1,Monsieur,Aubelle Flavien,Présent,174905,2007,1,NaN,NaN,NaN
2,Monsieur,Badoud Morgan,Présent,173922,2007,1,NaN,NaN,NaN
3,Monsieur,Baeriswyl Jonathan,Présent,179406,2007,1,NaN,NaN,NaN
4,Monsieur,Barroco Michael,Présent,179428,2007,1,NaN,NaN,NaN
5,Monsieur,Belfis Nicolas,Présent,179324,2007,1,NaN,NaN,NaN
6,Monsieur,Beliaev Stanislav,Présent,174597,2007,1,NaN,NaN,NaN
7,Monsieur,Bindschaedler Vincent,Présent,179449,2007,1,NaN,NaN,NaN
8,Monsieur,Bloch Marc-Olivier,Présent,178553,2007,1,NaN,NaN,NaN
9,Monsieur,Bloch Remi,Présent,179426,2007,1,NaN,NaN,NaN
